In [1]:
import serial
import time
import math
import subprocess
import os

In [4]:
# Serial port
stream = os.popen('powershell -command "[System.IO.Ports.SerialPort]::getportnames()"')
output = stream.read()[:-1] #return output of used usb, if no others are connected
print(output)

ser = serial.Serial(output) # Open serial port which is connected to the device
ser.baudrate = 57600
print("Serial port used: " + ser.name) # Check which port was really used

SerialException: could not open port '': FileNotFoundError(2, 'The system cannot find the path specified.', None, 3)

In [3]:
ser

Serial<id=0x1f00f6cca30, open=True>(port='COM5', baudrate=57600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [6]:
import binascii
binascii.unhexlify(b'128')

Error: Odd-length string

# Byte Analysis

In [14]:
###testing "wizard" sequence

#string='ÿ°Øð'
#a=string.encode()
#ser.write(b"\x1b\x00\x20\x3a\x8d\x57\x89\xc5\xff\xff\x00\x00\x00\x00\x09\x00\x01\x01\x00\x01\x00\x81\x03\x0d\x00\x00\x00\x01\x60\x0f\x11\x10\x05\x80\xa4\x02\x00\xf9\x45\xf0")
ser.write(b"\x0f\x10\xff\x05\xb0\x01\x02\x00\x11\xd8\xf0")
time.sleep(0.2)
ser.read_all()


b'\x0f\x11\x10\x03\xb0\x01\x00\xd5\xf0'

In [10]:
# LDEna - Laser driver enable
print("\nLDEna - Laser driver enable:")
b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
#b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
#this is the byte architecture
#Start of frame, Source Address, Destination Address, Length of data,[...Data...],Checksum, End of frame
#Data: Alarm-> CLDD temp. fatal threshold, System Register-> Laser Driver enable, No Op
ser.write(b)

time.sleep(0.5) # Wait 500 ms

i = 0
while(i < 9):
    i = i + 1
    byte = ser.read(1) # Read data byte
    print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))

time.sleep(3.000) # Wait 3 s


LDEna - Laser driver enable:
byte (bin) = 0b00001111
byte (bin) = 0b00010001
byte (bin) = 0b00010001
byte (bin) = 0b00000011
byte (bin) = 0b01000000
byte (bin) = 0b00010101
byte (bin) = 0b00000000
byte (bin) = 0b01111010
byte (bin) = 0b11110000


In [5]:
# DevType - Device type
print("\nDevType - Device type:")
b=b"\x0F\x11\xFF\x03\x80\x07\x00\x9A\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

ser.read_all()
#i = 0
#while(i < 21):
   # i = i + 1
   # byte = ser.read(1) # Read data byte
   # print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


DevType - Device type:


b"\x0f\x11\x11\x0f\x80\x07\x0cLASER DRIVER'\xf0"

In [9]:
# PLIPSet - Configures the laser diode current/power (write)
print("\nPLIPSet - Configures the laser diode current/power (write):")
#b=b"\x0f\x10\x11\x05\xb0\x72\x02\x00\xfb\x45\xf0"
def string_to_byte(x):
    if x<=197:
        h1=hex(x)[2:]
        h2=hex(145-(100-x))[2:]
        stringo="0F11FF07A0720400" + h1 + "0000" + h2 + 'F0'
        return bytes.fromhex(stringo)
    else:
        h1=hex(x)[2:]
        h2=hex(69-(251-x))[2:]
        stringo="0F101105B0720200" + h1 + h2 + 'F0'
        return bytes.fromhex(stringo)
    
#b=b"\x0F\x11\xFF\x07\xA0\x72\x04\x00\x32\x00\x00\x5F\xF0" #0x64=100

b=string_to_byte(50)
ser.write(b)
#this is the byte architecture
#Start of frame, Source Address, Destination Address, Length of data,[...Data...],Checksum, End of frame
#Data: Monitorization->CLDD temp.,Current/Power config, create lengthed buffer, buffer...
time.sleep(0.500) # Wait 500 ms

print(ser.read_all())

#i = 0
#while(i < 9):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
   # print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


PLIPSet - Configures the laser diode current/power (write):
b'\x0f\x11\x11\x03\xa0r\x007\xf0'


In [8]:
# PLIPSet - Configures the laser diode current/power (read)
print("\nPLIPSet - Configures the laser diode current/power (read):")
b=b"\x0F\x11\xFF\x03\x80\xA1\x00\x34\xF0" #current
#data: laser output power, config diode current/power, No op
ser.write(b)

time.sleep(0.500) # Wait 500 ms

a=ser.read_all()
print(a)
print(a[-3])
#i = 0
#while(i < 11):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
   # if (i==9 or i==10):
    #    print(int(byte[1:], base=16),byte)


PLIPSet - Configures the laser diode current/power (read):
b'\x0f\x11\x11\x03D\x13\x00|\xf0\x0f\x11\x11\x03@\x14\x00y\xf0\x0f\x11\x11\x05\x80\xa1\x02\x00\x00J\xf0'
0


In [17]:
# TECTSet - Configures the TEC temperature (write)
print("\nTECTSet - Configures the TEC temperature (write):")
#b=b"\x0F\x11\xFF\x06\xA0\x75\x03\x00\xC8\x00\xF6\xF0"
#b=b"\x0f\x10\x11\x05\xb0\x75\x02\x00\xfb\x48\xf0"

def string_to_byte1(x):
    h1=hex(x)[2:]
    h2=hex(72-(251-x))[2:]
    print(h1,h2)
    stringo="0F101105B0750200" + h1 + h2 + 'F0'
    print(stringo)
    return bytes.fromhex(stringo)

#C8=200
b=string_to_byte1(250) #ºC*10
ser.write(b)
time.sleep(0.500) # Wait 500 ms

ser.read_all()
#i = 0
#while(i < 9):
#    i = i + 1
 #   byte = ser.read(1) # Read data byte
  #  print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


TECTSet - Configures the TEC temperature (write):
fa 47
0F101105B0750200fa47F0


b'\x0f\x11\x10\x03\xb0u\x00I\xf0'

In [3]:
# TECTSet - Configures the TEC temperature (read)
print("\nTECTSet - Configures the TEC temperature (read):")
b=b"\x0F\x11\xFF\x03\x80\x75\x00\x08\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

a=ser.read_all()
print(a[-3])
print(a[-2])

#i = 0
#while(i < 11):
 #   i = i + 1
  #  byte = ser.read(1) # Read data byte
     #print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))
b=b"\x0F\x11\xFF\x03\x80\xA4\x00\x37\xF0" #this is the real controller
ser.write(b)

time.sleep(0.500) # Wait 500 ms

a=ser.read_all()
print(a[-3])
print(a[-2])


TECTSet - Configures the TEC temperature (read):
250
24
251
72


In [15]:
# LDDis - Laser driver disable
print("\nLDDis - Laser driver disable:")
b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
#b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
ser.write(b)

time.sleep(0.500) # Wait 500 ms

#print(ser.read_all())
i = 0
while(i < 9):
    i = i + 1
    byte = ser.read(1) # Read data byte
    print("byte (bin) = 0b" + "{0:08b}".format(ord(byte)))


LDDis - Laser driver disable:
byte (bin) = 0b00001111
byte (bin) = 0b00010001
byte (bin) = 0b00010001
byte (bin) = 0b00000011
byte (bin) = 0b01000000
byte (bin) = 0b00010110
byte (bin) = 0b00000000
byte (bin) = 0b01111011
byte (bin) = 0b11110000


In [7]:
#restart
b=b"\x0F\x11\xFF\x03\x40\x14\x00\x67\xF0"
ser.write(b)

9

In [3]:
#b=b"\x0F\x11\xFF\x06\xA0\x75\x03\x00\xC8\x00\xF6\xF0"
#b=b"\x0f\x10\x11\x05\xa0\x25\x02\x00\x00\xed\xf0"
#b=b"\x0f\x10\x11\x05\xb0\x75\x02\x00\xfa\x47\xf0"
#b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
#b=b"\x0f\x10\x11\x05\xb0\x72\x02\x00\xfb\x45\xf0"
b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
ser.write(b)
time.sleep(0.2)
status=''
a=ser.read_all()
print(a)
print(a[-3])
print(a[-2])
if a[-3]==36:
    status='Disabled'
if a[-3]==7:
    status='Hibernating'
if a[-3]==5:
    status='Enabled'
if a[-3]==2:
    status='Ready'

print(status)

b'\x0f\x11\x10\x05\x80\x10\x02\x00\x07\xbf\xf0'
7
191
Hibernating


In [5]:
# Serial port
ser.close() # Close serial port
print("\nSerial port closed")


Serial port closed


# Classe

In [1]:
import serial
import time
import os

stream = os.popen('powershell -command "[System.IO.Ports.SerialPort]::getportnames()"')
output = stream.read()[:-1] #return output of used usb, if no others are connected

class mwLaser():
    
    def __init__(self):
        super().__init__()
        
        self.port = serial.Serial(output) # Open serial port which is connected to the device
        self.port.baudrate = 57600
        self.current = 0
        self.power = 0
        self.TEC_temperature = 0
        self.status='Ready'
        self.laser_status()
        
    def laser_status(self):
        time.sleep(0.3)
        b=b"\x0f\x10\x11\x05\x80\x10\x02\x00\x00\xb8\xf0"
        self.port.write(b)
        time.sleep(0.2)
        statuse=''
        a=self.port.read_all()
        print(a)
        dig= a[-3] if len(str(a[-2]))>len(str(a[-3])) else a[-2]
        if dig in [36,66,28]:
            statuse='Disabled'
        if dig in [7,71]:
            statuse='Hibernating'
        if dig in [5,53,37,117,45]:
            statuse='Active'
        if dig in [2]:
            statuse='Ready'
        if dig in [67,69,29]:
            statuse='Configuring'
        self.status=statuse
        
    def update_current(self):
        b=b"\x0F\x11\xFF\x03\x80\xA1\x00\x34\xF0"
        self.port.write(b)
        time.sleep(0.200)
        self.current=self.port.read_all()[-3]
        
    def update_power(self):
        b=b"\x0F\x11\xFF\x03\x80\xA2\x00\x35\xF0"
        self.port.write(b)
        time.sleep(0.200)
        self.power=self.port.read_all()[-3]
    
    def update_TEC_temperature(self):
        b=b"\x0F\x11\xFF\x03\x80\xA4\x00\x37\xF0"
        self.port.write(b)
        time.sleep(0.200)
        self.TEC_temperature=self.port.read_all()[-3]/10
    
    def update_TEC_current(self):
        a=0
    
    def set_TEC_temperature(self,x):
        h1=hex(x)[2:]
        h2=hex(246-(200-x))[2:]
        stringo="0F101105B0750200" + h1 + h2 + 'F0'
        b=bytes.fromhex(stringo)
        self.port.write(b)
        time.sleep(0.200)
        
    def set_current(self,x):
        if x<=197:
            h1=hex(x)[2:]
            h2=hex(145-(100-x))[2:]
            stringo="0F11FF07A0720400" + h1 + "0000" + h2 + 'F0'
            b=bytes.fromhex(stringo)
            self.port.write(b)
            time.sleep(0.200)
        else:
            h1=hex(x)[2:]
            h2=hex(69-(251-x))[2:]
            stringo="0F101105B0720200" + h1 + h2 + 'F0'
            b=bytes.fromhex(stringo)
            self.port.write(b)
            time.sleep(0.200)
    
    def return_data(self):
        update_current()
        update_power()
        update_TEC_temperature()
        laser_status()
        return [self.current,self.power,self.TEC_temperature,self.status]
    
    def enable(self):
        b=b"\x0F\x11\xFF\x03\x40\x15\x00\x68\xF0"
        self.port.write(b)
        
        
    def disable(self):
        b=b"\x0F\x11\xFF\x03\x40\x16\x00\x69\xF0"
        self.port.write(b)
    
    def reset(self):
        time.sleep(0.100)
        b=b"\x0F\x11\xFF\x03\x40\x14\x00\x67\xF0"
        self.port.write(b)
        
    def hibernate(self):
        self.disable()
        time.sleep(0.100)
        b=b"\x0F\x11\xFF\x03\x40\x13\x00\x66\xF0"
        self.port.write(b)
    
    def close_port(self):
        self.port.close()
        print("\nSerial port closed")

In [2]:
laser=mwLaser()

b''


IndexError: index out of range

In [3]:
#laser.enable()
#laser.laser_status()
laser.status

'Active'

In [8]:
laser.set_current(50)
#laser.reset()

In [18]:
laser.disable()

In [15]:
laser.close_port()


Serial port closed
